In [21]:
import numpy as np
import os
import sys
from tqdm import tqdm
import pdb;pdb.set_trace=lambda:None

# contact with humans bbox.
### livingroom
# contact_dir = '/ps/project/scene_generation/datasets/preprocess_contact/interaction'
# static_human_free_dir = '/is/cluster/scratch/scene_generation/datasets/preprocess_freespace_staticHumans/free_space'
# pingpong_human_free_dir = '/is/cluster/scratch/scene_generation/datasets/preprocess_freespace/free_space'
# output_dir = '/ps/project/scene_generation/datasets/preprocess_fuse'

### bedroom
# contact_dir = '/ps/project/scene_generation/datasets/preprocess_contact/interaction'
# static_human_free_dir = '/is/cluster/scratch/scene_generation/datasets/preprocess_freespace_staticHumans/free_space'
# pingpong_human_free_dir = '/is/cluster/scratch/scene_generation/datasets/preprocess_freespace/free_space'
# output_dir = '/ps/project/scene_generation/datasets/preprocess_fuse'

### library
contact_dir = '/is/cluster/scratch/scene_generation/datasets/prepare_inputs/library_contact_debug/interaction'
static_human_free_dir = '/is/cluster/scratch/scene_generation/datasets/prepare_inputs/library_freespace_staticHumans/free_space'
pingpong_human_free_dir = '/is/cluster/scratch/scene_generation/datasets/prepare_inputs/library_freespace/free_space/'
output_dir = '/is/cluster/scratch/scene_generation/datasets/prepare_inputs/library_preprocess_fuse'

### dining_room
# contact_dir = '/is/cluster/scratch/scene_generation/datasets/prepare_inputs/diningroom_contact/interaction'
# static_human_free_dir = '/is/cluster/scratch/scene_generation/datasets/prepare_inputs/diningroom_freespace_staticHumans/free_space'
# pingpong_human_free_dir = '/is/cluster/scratch/scene_generation/datasets/prepare_inputs/diningroom_freespace/free_space'
# output_dir = '/is/cluster/scratch/scene_generation/datasets/prepare_inputs/diningroom_preprocess_fuse'

In [22]:
contact_list = os.listdir(contact_dir)
static_free_list = os.listdir(static_human_free_dir)
pingpong_free_list = os.listdir(pingpong_human_free_dir)


# Python program to illustrate the intersection
# of two lists using set() method
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

useful_list1 = intersection(static_free_list, pingpong_free_list)
useful_list = intersection(contact_list, useful_list1)

In [23]:
print('contact_list: ', len(contact_list))
print('static_free_list: ', len(static_free_list))
print('pingpong_free_list: ', len(pingpong_free_list))
print(len(useful_list))
print(len(contact_list))
print(len(pingpong_free_list))
print(len(static_free_list))

print(set(pingpong_free_list) - set(static_free_list))

contact_list:  624
static_free_list:  624
pingpong_free_list:  624
624
624
624
624
set()


In [24]:
import numpy as np
def check_has_bbox_npz(input_path):
    return os.path.exists(os.path.join(input_path, 'boxes.npz'))

def load_npz(input_path):
    data = np.load(input_path, allow_pickle=True)
    return data

def save_npz(data, out_path):
    np.savez_compressed(out_path, **data)

def merge_pingpong_masks(free_space_split):
    filled_body_free_space_aug_1 = free_space_split
    merge_bodies = []
    for i in range(filled_body_free_space_aug_1.shape[-1]):
        tmp_merge = np.any(filled_body_free_space_aug_1[:, :, :i+1], -1) * 255.0
        tmp_merge = tmp_merge.astype(np.uint8)
        merge_bodies.append(tmp_merge[:, :,None])
        # if debug:
        #     tmp_img = Image.fromarray(tmp_merge)
        #     tmp_img.save(os.path.join(save_dir, f'{i:03d}.png'))

    return np.concatenate(merge_bodies, -1)

In [25]:
def merge(contact_dir, static_free_dir, pingpong_free_dir, output_dir, useful_list):
    cnt = 0
    useless_list = []
    for file_t in tqdm(useful_list):
        if '.txt' in file_t:
            continue
        try:
        # if True:
            file = file_t + '/' + 'boxes.npz'
            contact_file = os.path.join(contact_dir, file)
            static_free_file = os.path.join(static_free_dir, file)
            pingpong_free_file = os.path.join(pingpong_free_dir, file)

            output_file = os.path.join(output_dir, file)
            tmp_output_dir = os.path.dirname(output_file)

            import pdb;pdb.set_trace()
            # print('output_file: ', output_file)
#             if os.path.exists(output_file):
#                 print('exists', output_file)
#                 continue
            if not os.path.exists(tmp_output_dir):
                os.makedirs(tmp_output_dir, exist_ok=True)

            # import pdb;pdb.set_trace()
            contact_data = dict(np.load(contact_file, allow_pickle=True))
            static_free_data = dict(np.load(static_free_file, allow_pickle=True))
            pingpong_free_data = dict(np.load(pingpong_free_file, allow_pickle=True))

            ### fuse two free space
            filled_body_free_space_aug_1 = static_free_data['filled_body_free_space_aug']
            filled_body_free_space_aug_2 = pingpong_free_data['filled_body_free_space_aug']

            ### merge pingpong masks.
            filled_body_free_space_aug_2 = merge_pingpong_masks(filled_body_free_space_aug_2)
            
            import pdb;pdb.set_trace()
            # check mask range.
            if filled_body_free_space_aug_1.max() == 1.0:
                filled_body_free_space_aug_1 = (filled_body_free_space_aug_1 * 255).astype(np.uint8)
            if filled_body_free_space_aug_2.max() == 1.0:
                filled_body_free_space_aug_2 = (filled_body_free_space_aug_2 * 255).astype(np.uint8)

            fuse_body_free_space = np.concatenate([filled_body_free_space_aug_1, 
                filled_body_free_space_aug_2], -1)

            ### final free space data.
            static_free_data['filled_body_free_space_aug'] = fuse_body_free_space

            import pdb;pdb.set_trace()

            # print(contact_data.keys())
            merge_keys = ['filled_body_free_space', 'filled_body_idx', 'filled_body_free_space_aug',
                  'filled_body_sample_path'] #'filled_body_idx_aug',
            for key in merge_keys:
                if key in contact_data.keys():
                    # print(key)
                    contact_data['free'+key] = static_free_data[key]
                else:
                    contact_data[key] = static_free_data[key]

            # print(f'save to {cnt}', output_file)
            # break;
            # import pdb;pdb.set_trace()
            # output_data = np.concatenate((contact_data, free_data), axis=0)
            np.savez_compressed(output_file, **contact_data)
            cnt += 1
        except:
            useless_list.append(file_t)

        print('useful, cnt: ',  cnt, '/', len(useless_list)+cnt)
    print(useless_list)
    return useless_list

In [26]:
merge(contact_dir, static_human_free_dir, pingpong_human_free_dir, output_dir, useful_list)

useful, cnt:  1 / 1


useful, cnt:  2 / 2


useful, cnt:  3 / 3


useful, cnt:  4 / 4


useful, cnt:  5 / 5


...

useful, cnt:  619 / 619


useful, cnt:  620 / 620


useful, cnt:  621 / 621


useful, cnt:  622 / 622
[]


[]

In [27]:
print(len(useless_list))

NameError: name 'useless_list' is not defined